In [ ]:
"""
Purpose: To Run the neuron preprocessing


"""

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

# configuring the virtual module

In [3]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 30)
print(f"Sleeping {random_sleep_sec} sec before conneting")
#time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-22 21:09:52,160 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-22 21:09:52,162 - settings - Setting database.user to celiib
INFO - 2020-11-22 21:09:52,163 - settings - Setting database.password to newceliipass
INFO - 2020-11-22 21:09:52,175 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-22 21:09:52,176 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 21:09:52,189 - connection - Connect

Sleeping 8 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
Connecting celiib@at-database.ad.bcm.edu:3306


# Defining Our Table

In [4]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [5]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 4498


In [6]:
#so that it will have the adapter defined
from datajoint_utils import *

In [ ]:
#minnie.Decomposition.drop()
#schema.external['decomposition'].delete(delete_external_files=True)

In [ ]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branche

    
    """

    key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0").proj())
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n----- Working on {segment_id}-------")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        somas = du.get_soma_mesh_list(segment_id) 
        print(f"somas = {somas}")
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")



        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Cleaning the external files

In [ ]:
(minnie.schema.jobs & "table_name='__decomposition'").delete()
#((schema.jobs & "table_name = '__decomposition'") & "timestamp>'2020-11-16 00:26:00'").delete()

In [ ]:
import preprocessing_vp2 as pre
import compartment_utils as cu
cu = reload(cu)
pre = reload(pre)

In [ ]:
import time
import random

start_time = time.time()
#time.sleep(random.randint(0, 900))
print('Populate Started')
Decomposition.populate(reserve_jobs=True, suppress_errors=False, order='random')
print('Populate Done')

print(f"Total time for Decomposition populate = {time.time() - start_time}")

# Clearing Files

In [ ]:
curr_path = Path("/mnt/dj-stor01/platinum/minnie65/02/decomposition/")
for f in curr_path.iterdir():
    print(f'{f}')
    #f.unlink()
    

# Checking the Table

In [ ]:
minnie.Decomposition()

In [ ]:
du = reload(du)
from datajoint_utils import *
my_dj_neuron = (minnie.Decomposition & "segment_id=864691134221889045").fetch1("decomposition")

import neuron_visualizations as nviz
nviz.visualize_neuron(my_dj_neuron,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")

# Debugging the neuron creation

In [7]:
#1) Get the segment id from the key
segment_id = 864691136309663834
description = "0_25"
print(f"\n\n----- Working on {segment_id}-------")
global_start = time.time()

#2) Get the decimated mesh
current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

#3) Get the somas info *************************** Need to change this when actually run *******************
somas = du.get_soma_mesh_list(segment_id) 
print(f"somas = {somas}")
#4) Run the preprocessing


total_neuron_process_time = time.time()

INFO - 2020-11-22 21:10:20,207 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 21:10:20,379 - settings - Setting enable_python_native_blobs to True




----- Working on 864691136309663834-------


INFO - 2020-11-22 21:11:02,425 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 21:11:02,626 - settings - Setting enable_python_native_blobs to True


somas = [[<trimesh.Trimesh(vertices.shape=(1079, 3), faces.shape=(1651, 3))>], array([418.5506]), array([0.445])]


In [8]:
import preprocessing_vp2 as pre
import soma_extraction_utils as sm

import neuron
tu = reload(tu)
du = reload(du)
sm = reload(sm)
pre = reload(pre)
import compartment_utils as cu
cu = reload(cu)

neuron = reload(neuron)
print(f"\n--- Beginning preprocessing of {segment_id}---")
recovered_neuron = neuron.Neuron(
mesh = current_neuron_mesh,
somas = somas,
segment_id=segment_id,
description=description,
suppress_preprocessing_print=False,
suppress_output=False,
calculate_spines=True,
widths_to_calculate=["no_spine_median_mesh_center"]

        )

INFO - 2020-11-22 21:11:19,293 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-22 21:11:19,294 - settings - Setting database.user to celiib
INFO - 2020-11-22 21:11:19,295 - settings - Setting database.password to newceliipass



--- Beginning preprocessing of 864691136309663834---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(1079, 3), faces.shape=(1651, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(1079, 3), faces.shape=(1651, 3))>]
soma_mesh_list_centers = [array([1287719.97590361,  686776.72363299,  718608.14698795])]
xvfb-run -n 4498 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Neuron_Processing/temp/neuron_21410.off -o /notebooks/Platinum_Datajoint/Neuron_Processing/temp/neuron_21410_fill_holes.off -s /notebooks/Platinum_Datajoint/Neuron_Processing/temp/fill_holes_294087.mls

---- meshlab output -----
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
Current Plugins Dir is: /meshlab/src/distrib/plugins 
Error While parsing the XML filter plugin descripto

  0%|          | 0/1 [00:00<?, ?it/s]

Total time for mesh_pieces_connectivity= 0.122650146484375
# of insignificant_limbs = 1 with trimesh : [<trimesh.Trimesh(vertices.shape=(364, 3), faces.shape=(546, 3))>]
# of not_processed_soma_containing_meshes = 0 with trimesh : []


----- Working on Proper Limb # 0 ---------
meshparty_segment_size = 100
Time for preparing soma vertices and root: 4.0531158447265625e-06
cc_vertex_thresh = 10


100%|██████████| 1/1 [00:00<00:00, 19.91it/s]

...




--- Working on 1-to-1 correspondence-----
max(original_labels),len(original_labels) = (1, 2)
empty_indices % = 0.19771501110758488
 conflict_indices % = 0.004125674389082831



AFTER face_lookup_resolved_test



main_branch = 85
The closest float distance was 9366.733923516935 which was greater than the maximum stitch distance 8000
 --> so ending the floating mesh stitch processs
Total time for stitching floating pieces = 140.38363027572632


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [1286318.      682395.3875  717834.275 ]---------
Starting_edge inside branches_to_conept = [[1284808.11035883  682321.35797433  717654.99169998]
 [1286318.          682395.3875      717834.275     ]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [82]
printing out current edge:
[[1286318.          682395.3875      717834.275     ]
 [1284808.11035883  682321.35797433  717654.99169998]]
edge_endpoints_to_process was empty so exiting loop after 172 iterations
starting_node in concept map (that should match the starting edge) = 2
Total time for branches to concept conversion = 1.2664635181427002

Done generating concept network 


recovered_touching

Assigning the old width calculation because no valid new widths
Working on limb L0 branch 67
Assigning the old width calculation because no valid new widths
Working on limb L0 branch 118
Assigning the old width calculation because no valid new widths
Working on limb L0 branch 90
Assigning the old width calculation because no valid new widths
Working on limb L0 branch 72
Assigning the old width calculation because no valid new widths
Working on limb L0 branch 66
Working on limb L0 branch 69
Working on limb L0 branch 119
Assigning the old width calculation because no valid new widths
Working on limb L0 branch 91
Assigning the old width calculation because no valid new widths
Working on limb L0 branch 73
Working on limb L0 branch 65
Assigning the old width calculation because no valid new widths
Working on limb L0 branch 68
Working on limb L0 branch 76
Assigning the old width calculation because no valid new widths
Working on limb L0 branch 120
Assigning the old width calculation because 


Working on limb L0 branch 2
No spines and using precomputed width
Working on limb L0 branch 0
No spines and using precomputed width
Working on limb L0 branch 1
No spines and using precomputed width
Working on limb L0 branch 6
No spines and using precomputed width
Working on limb L0 branch 3
No spines and using precomputed width
Working on limb L0 branch 4
No spines and using precomputed width
Working on limb L0 branch 7
No spines and using precomputed width
Working on limb L0 branch 5
No spines and using precomputed width
Working on limb L0 branch 10
No spines and using precomputed width
Working on limb L0 branch 11
No spines and using precomputed width
Working on limb L0 branch 13
No spines and using precomputed width
Working on limb L0 branch 8
No spines and using precomputed width
Working on limb L0 branch 9
No spines and using precomputed width
Working on limb L0 branch 12
No spines and using precomputed width
Working on limb L0 branch 14
No spines and using precomputed width
Work

In [ ]:
import neuron_visualizations as nviz

In [ ]:
nviz.visualize_neuron(recovered_neuron,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")

In [ ]:
"/note"